In [26]:
import pandas as pd
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller

In [27]:
df = pd.read_csv('concatenated_TQQQ_2019_2023.csv')

In [28]:
df

,timestamp,open,high,low,close,volume,vwap
0,2019-01-02 04:00:00-05:00,8.6750,8.675,8.6725,8.6725,4784.0,8.6746
1,2019-01-02 04:01:30-05:00,8.6250,8.625,8.5950,8.5950,11776.0,8.6021
2,2019-01-02 04:02:00-05:00,8.5950,8.595,8.5950,8.5950,2000.0,8.5950
3,2019-01-02 04:02:30-05:00,8.5950,8.595,8.5950,8.5950,800.0,8.5950
4,2019-01-02 04:02:45-05:00,8.5950,8.595,8.5950,8.5950,800.0,8.5950
...,...,...,...,...,...,...,...
3145860,2023-12-01 19:58:15-05:00,44.4300,44.430,44.4300,44.4300,430.0,44.4300
3145861,2023-12-01 19:58:45-05:00,44.4200,44.420,44.4200,44.4200,709.0,44.4200
3145862,2023-12-01 19:59:15-05:00,44.4299,44.430,44.4299,44.4300,2303.0,44.4300
3145863,2023-12-01 19:59:30-05:00,44.4300,44.430,44.4300,44.4300,100.0,44.4300


In [29]:
def SMA(data, window_size):
    return data['close'].rolling(window=window_size).mean()

def EMA(data, window_size):
    return data['close'].ewm(span=window_size).mean()

def MACD(data, short_window, long_window):
    short_EMA = EMA(data, short_window)
    long_EMA = EMA(data, long_window)
    return short_EMA - long_EMA

def RSI(data, window_size):
    delta = data['close'].diff()
    delta = delta[1:] 
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=window_size-1 , min_periods=window_size).mean()
    ema_down = down.ewm(com=window_size-1 , min_periods=window_size).mean()
    return ema_up/ema_down

def Bollinger_Bands(data, window_size):
    middle_band = SMA(data, window_size)
    std_dev = data['close'].rolling(window=window_size).std()
    upper_band = middle_band + (std_dev*2)
    lower_band = middle_band - (std_dev*2)
    return upper_band, lower_band

In [30]:
def check_stationarity(series):
    # Thanks to https://machinelearningmastery.com/time-series-data-stationary-python/

    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

In [31]:
def date_features(df):
    # Create time series features based on time series index.
    df.index = pd.to_datetime(df.index)
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

In [32]:
# check_stationarity(df.close)
# check_stationarity(df.close.tail(1000))

In [33]:
# check_stationarity(df.close.tail(10000).diff(periods=1).dropna())

In [34]:
df["close_diff_1"] = df.close.diff(periods=1)

In [35]:

df.index = df["timestamp"]
df.drop("timestamp", axis=1, inplace=True)
df = date_features(df)

C:\Users\TGAdmin\AppData\Local\Temp\ipykernel_4524\4007242149.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df.index = pd.to_datetime(df.index)


AttributeError: 'Index' object has no attribute 'dayofweek'